# Evaluate healthcare facility development cost predictability


## Higher delivery costs and longer timelines then other facilities, particularly in California

#### Strict and actively monitored seismic, health, fire life-safety regulations (OSHPD and CDPH)

#### Inpatient facilities required tailored services for complex, specialized medical programs

#### Limited supply of qualified designers, contractors, and specialized consultants 

#### Large cost variation tied to national and local market dynamics as well delivery risks in older campuses and facilities

#### Declining provider margins limit capital investment while end user appetites and expectations remain high


## Approach overview

#### Applied multiple linear regression to evaluate relationship by construction cost and delivery duration

#### First evaluated projects as one group with a categorical value and separated projects types

#### Created visualizations in Tableau to evaluate data and predict costs based on data



In [2]:
# Dependencies 
import pandas as pd
import datetime
import numpy as np
import os
import csv
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# Import and re CA Office of State Planning and Development (OSHPD) closed hospital projects 
# and Federal Reserve Economic Data (FRED) Non-residential construction Producer Price Index
Hospital_Projects = pd.read_csv("../Resources/2019_07_16_Hospital_Closed_Project_Summary.csv", encoding='ISO-8859-1')
Construction_PPI = pd.read_csv("../Resources/FRED_NonResConstPPI_2012-2019.csv", encoding='ISO-8859-1')

## Data cleaning and preparation

#### Public data from OSHPD included project applications with info on facility, location, project, type, dates, size, costs

#### Data includes 30,753 rows (2001 to 2019 completed projects) and required significant cleaning and preparation

#### Joined OSHPD data with Federal Reserve national nonresidential construction PPI data to escalate historic costs to 2019

#### Eliminated duplicative rows and duplicative or unnecessary columns

#### Converted data types and created duration in months with the provided project submittal and close dates

#### Created new columns with correct data and cost per square foot metrics

#### Utilized Tableau to review data throughout to identify and address shortcomings; reduced to 3946 rows between 2011 to 2019

#### Filtered projects by size (>=100 SF), cost (>=$100K), and type (Maintenance, Remodels, Additions, New Buildings)

#### Exported cleaned csv to Machine Learning Notebook

In [157]:
Hospital_Projects.head()

FacilityNbr      FacilityName        FacilityType       Address1 AddrCity  \
0        10006  Orchard Hospital  General Acute Care  240 Spruce St  Gridley   
1        10006  Orchard Hospital  General Acute Care  240 Spruce St  Gridley   
2        10006  Orchard Hospital  General Acute Care  240 Spruce St  Gridley   
3        10006  Orchard Hospital  General Acute Care  240 Spruce St  Gridley   
4        10006  Orchard Hospital  General Acute Care  240 Spruce St  Gridley   

  AddrState AddrZip FacilityStatus   Latitude  Longitude  \
0        CA   95948         Opened  39.367691 -121.68956   
1        CA   95948         Opened  39.367691 -121.68956   
2        CA   95948         Opened  39.367691 -121.68956   
3        CA   95948         Opened  39.367691 -121.68956   
4        CA   95948         Opened  39.367691 -121.68956   

            ...           ProjSubType  KindofProject ConstructionCosts  \
0           ...                   NaN        Remodel           5600.00   
1           ...                   NaN        Remodel           2000.00   
2           ...                   NaN      NullValue          18000.00   
3           ...                   NaN        Remodel         222715.39   
4           ...                   NaN       Addition         112782.00   

  RADCosts FixedEquipmentCosts TotalCosts    FileDate  \
0      0.0                 0.0    5600.00   7/19/2001   
1      0.0                 0.0    2000.00  12/13/2001   
2      0.0                 0.0   18000.00   1/26/2005   
3      0.0                 0.0  222715.39   5/20/2010   
4      0.0                 0.0  112782.00   4/18/2012   

                 ProjStatus  ProjStatusDate  SquareFootageofProject  
0                    Closed       6/27/2002                       0  
1                    Closed      12/13/2002                       0  
2  Closed Non-CAC Compliant      12/21/2007                       0  
3                    Closed        4/3/2012                       0  
4                    Closed       5/19/2014                       1  

[5 rows x 25 columns]

In [158]:
#Identify full count of projects
Hospital_Projects.count()

FacilityNbr               30753
FacilityName              30753
FacilityType              30753
Address1                  30753
AddrCity                  30753
AddrState                 30753
AddrZip                   30753
FacilityStatus            30753
Latitude                  30753
Longitude                 30753
ProjectNbr                30753
FacilityNbr.1             30753
ProjectName               30750
Description               30753
ProjType                  30753
ProjSubType                 145
KindofProject             30753
ConstructionCosts         30753
RADCosts                  30753
FixedEquipmentCosts       30753
TotalCosts                30753
FileDate                  30753
ProjStatus                30753
ProjStatusDate            30753
SquareFootageofProject    30753
dtype: int64

In [159]:
#Identify the data types of the row values
Hospital_Projects.dtypes

FacilityNbr                 int64
FacilityName               object
FacilityType               object
Address1                   object
AddrCity                   object
AddrState                  object
AddrZip                    object
FacilityStatus             object
Latitude                  float64
Longitude                 float64
ProjectNbr                 object
FacilityNbr.1               int64
ProjectName                object
Description                object
ProjType                   object
ProjSubType                object
KindofProject              object
ConstructionCosts         float64
RADCosts                  float64
FixedEquipmentCosts       float64
TotalCosts                float64
FileDate                   object
ProjStatus                 object
ProjStatusDate             object
SquareFootageofProject      int64
dtype: object

In [160]:
Construction_PPI.head()

DATE  WPU801_NBD20190601  Year  Month Year_Month
0   6/1/2012            82.83881  2012      6     2012-6
1   7/1/2012            82.83881  2012      7     2012-7
2   8/1/2012            82.91901  2012      8     2012-8
3   9/1/2012            82.83881  2012      9     2012-9
4  10/1/2012            82.99920  2012     10    2012-10

In [161]:
# Check value data types
Construction_PPI.dtypes

DATE                   object
WPU801_NBD20190601    float64
Year                    int64
Month                   int64
Year_Month             object
dtype: object

In [162]:
#Dropping columns with unnecessary values and check number of rows.  
#Also Dropping "TotalCosts" column because not consistently add up with other cost columns
Hospital_Projects_1 = Hospital_Projects.drop(columns=["FacilityStatus", "FacilityNbr.1", "ProjType", "FacilityType",
                                                      "ProjSubType", "ProjStatus", "TotalCosts"])
Hospital_Projects_1.count()

FacilityNbr               30753
FacilityName              30753
Address1                  30753
AddrCity                  30753
AddrState                 30753
AddrZip                   30753
Latitude                  30753
Longitude                 30753
ProjectNbr                30753
ProjectName               30750
Description               30753
KindofProject             30753
ConstructionCosts         30753
RADCosts                  30753
FixedEquipmentCosts       30753
FileDate                  30753
ProjStatusDate            30753
SquareFootageofProject    30753
dtype: int64

In [163]:
#Identify any duplicative projects by Project Number
Hospital_Projects_1Dups=Hospital_Projects_1[Hospital_Projects['ProjectNbr'].duplicated(keep=False)]
Hospital_Projects_1Dups

FacilityNbr                                       FacilityName  \
2599         10362                 Queen Of The Valley Medical Center   
2600         10362                 Queen Of The Valley Medical Center   
4814         10554         Parkview Community Hospital Medical Center   
4815         10554         Parkview Community Hospital Medical Center   
9963         11059                       Santa Rosa Memorial Hospital   
9964         11059                       Santa Rosa Memorial Hospital   
10712        11164               Los Robles Hospital & Medical Center   
10713        11164               Los Robles Hospital & Medical Center   
11622        11338   Alta Bates Summit Medical Center - Summit Campus   
11623        11338   Alta Bates Summit Medical Center - Summit Campus   
14418        11809                     LAC/Harbor-UCLA Medical Center   
14419        11809                     LAC/Harbor-UCLA Medical Center   
19119        12359  UC San Diego Health Hillcrest - Hillcrest Medi...   
19120        12359  UC San Diego Health Hillcrest - Hillcrest Medi...   
19189        12359  UC San Diego Health Hillcrest - Hillcrest Medi...   
19190        12359  UC San Diego Health Hillcrest - Hillcrest Medi...   
21093        12476                                UCSF Medical Center   
21094        12476                                UCSF Medical Center   
21648        12662                          Eisenhower Medical Center   
21649        12662                          Eisenhower Medical Center   

                   Address1       AddrCity AddrState AddrZip   Latitude  \
2599        1000 Trancas St           Napa        CA   94558  38.324688   
2600        1000 Trancas St           Napa        CA   94558  38.324688   
4814    3865 Jackson Street      Riverside        CA   92503  33.927975   
4815    3865 Jackson Street      Riverside        CA   92503  33.927975   
9963     1165 Montgomery Dr     Santa Rosa        CA   95405  38.443710   
9964     1165 Montgomery Dr     Santa Rosa        CA   95405  38.443710   
10712   215 West Janss Road  Thousand Oaks        CA   91360  34.207592   
10713   215 West Janss Road  Thousand Oaks        CA   91360  34.207592   
11622    3100 Summit Street        Oakland        CA   94609  37.819901   
11623    3100 Summit Street        Oakland        CA   94609  37.819901   
14418  1000 West Carson St.       Torrance        CA   90502  33.829800   
14419  1000 West Carson St.       Torrance        CA   90502  33.829800   
19119      200 W. Arbor Dr.      San Diego        CA   92103  32.753822   
19120      200 W. Arbor Dr.      San Diego        CA   92103  32.753822   
19189      200 W. Arbor Dr.      San Diego        CA   92103  32.753822   
19190      200 W. Arbor Dr.      San Diego        CA   92103  32.753822   
21093  505 Parnassus Avenue  San Francisco        CA   94143  37.763106   
21094  505 Parnassus Avenue  San Francisco        CA   94143  37.763106   
21648     39000 Bob Hope Dr  Rancho Mirage        CA   92270  33.764652   
21649     39000 Bob Hope Dr  Rancho Mirage        CA   92270  33.764652   

        Longitude     ProjectNbr  \
2599  -122.296707  S152006-28-00   
2600  -122.296707  S152006-28-00   
4814  -117.438469  S150278-33-00   
4815  -117.438469  S150278-33-00   
9963  -122.701378  S171095-49-00   
9964  -122.701378  S171095-49-00   
10712 -118.882400  S142270-56-00   
10713 -118.882400  S142270-56-00   
11622 -122.264359  S141462-01-00   
11623 -122.264359  S141462-01-00   
14418 -118.294746  S151642-19-00   
14419 -118.294746  S151642-19-00   
19119 -117.165144     GS041355-8   
19120 -117.165144     GS041355-8   
19189 -117.165144     GL071280-4   
19190 -117.165144     GL071280-4   
21093 -122.457822  S161097-38-00   
21094 -122.457822  S161097-38-00   
21648 -116.405556   P-2012-02541   
21649 -116.405556   P-2012-02541   

                                             ProjectName  \
2599   North and South Egress Stairway Fire Rating Up...   
2600   North and Sou

In [164]:
#Drop duplicative projects, using ProjectNbr
Hospital_Projects_1 = Hospital_Projects_1.drop_duplicates(subset='ProjectNbr', keep='first')
Hospital_Projects_1.count()

FacilityNbr               30743
FacilityName              30743
Address1                  30743
AddrCity                  30743
AddrState                 30743
AddrZip                   30743
Latitude                  30743
Longitude                 30743
ProjectNbr                30743
ProjectName               30740
Description               30743
KindofProject             30743
ConstructionCosts         30743
RADCosts                  30743
FixedEquipmentCosts       30743
FileDate                  30743
ProjStatusDate            30743
SquareFootageofProject    30743
dtype: int64

In [165]:
# Eliminate ROCC KindsofProjects, since not applicable

Hospital_Projects_1=Hospital_Projects_1[Hospital_Projects_1.KindofProject != 'ROCC']
Hospital_Projects_1.count()

FacilityNbr               30725
FacilityName              30725
Address1                  30725
AddrCity                  30725
AddrState                 30725
AddrZip                   30725
Latitude                  30725
Longitude                 30725
ProjectNbr                30725
ProjectName               30722
Description               30725
KindofProject             30725
ConstructionCosts         30725
RADCosts                  30725
FixedEquipmentCosts       30725
FileDate                  30725
ProjStatusDate            30725
SquareFootageofProject    30725
dtype: int64

In [166]:
#Convert date columns from object to date values

Hospital_Projects_1['FileDate']=Hospital_Projects_1['FileDate'].astype('datetime64[ns]')
Hospital_Projects_1['ProjStatusDate']=Hospital_Projects_1['ProjStatusDate'].astype('datetime64[ns]')
Hospital_Projects_1.head()

FacilityNbr      FacilityName       Address1 AddrCity AddrState AddrZip  \
0        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
1        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
2        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
3        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
4        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   

    Latitude  Longitude    ProjectNbr                     ProjectName  \
0  39.367691 -121.68956    GS011843-1             NPC 2 RETROFIT WORK   
1  39.367691 -121.68956    SS013265-0              NPC 2 BULK MED GAS   
2  39.367691 -121.68956    GS050215-1         TOILET FINISHES UPGRADE   
3  39.367691 -121.68956    SS100827-0  ENTRANCE/LOBBY RECONFIGURATION   
4  39.367691 -121.68956  P-2012-00835            Chiller Installation   

                                         Description KindofProject  \
0                                NPC 2 RETROFIT WORK       Remodel   
1                                 NPC 2 BULK MED GAS       Remodel   
2                            TOILET FINISHES UPGRADE     NullValue   
3                     ENTRANCE/LOBBY RECONFIGURATION       Remodel   
4  Addition of a new exterior pad mounted chiller...      Addition   

   ConstructionCosts  RADCosts  FixedEquipmentCosts   FileDate ProjStatusDate  \
0            5600.00       0.0                  0.0 2001-07-19     2002-06-27   
1            2000.00       0.0                  0.0 2001-12-13     2002-12-13   
2           18000.00       0.0                  0.0 2005-01-26     2007-12-21   
3          222715.39       0.0                  0.0 2010-05-20     2012-04-03   
4          112782.00       0.0                  0.0 2012-04-18     2014-05-19   

   SquareFootageofProject  
0                       0  
1                       0  
2                       0  
3                       0  
4                       1

In [167]:
# Create columns for Schedule Month Duration as float and convert to Month Duration as Integer
Hospital_Projects_1['DurationMonthsFloat'] = (Hospital_Projects_1['ProjStatusDate'] - Hospital_Projects_1[
                                        'FileDate'])/np.timedelta64(1, 'M')
Hospital_Projects_1['DurationMonths'] = Hospital_Projects_1['DurationMonthsFloat'].astype(int)
Hospital_Projects_1.head()

FacilityNbr      FacilityName       Address1 AddrCity AddrState AddrZip  \
0        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
1        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
2        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
3        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
4        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   

    Latitude  Longitude    ProjectNbr                     ProjectName  \
0  39.367691 -121.68956    GS011843-1             NPC 2 RETROFIT WORK   
1  39.367691 -121.68956    SS013265-0              NPC 2 BULK MED GAS   
2  39.367691 -121.68956    GS050215-1         TOILET FINISHES UPGRADE   
3  39.367691 -121.68956    SS100827-0  ENTRANCE/LOBBY RECONFIGURATION   
4  39.367691 -121.68956  P-2012-00835            Chiller Installation   

                                         Description KindofProject  \
0                                NPC 2 RETROFIT WORK       Remodel   
1                                 NPC 2 BULK MED GAS       Remodel   
2                            TOILET FINISHES UPGRADE     NullValue   
3                     ENTRANCE/LOBBY RECONFIGURATION       Remodel   
4  Addition of a new exterior pad mounted chiller...      Addition   

   ConstructionCosts  RADCosts  FixedEquipmentCosts   FileDate ProjStatusDate  \
0            5600.00       0.0                  0.0 2001-07-19     2002-06-27   
1            2000.00       0.0                  0.0 2001-12-13     2002-12-13   
2           18000.00       0.0                  0.0 2005-01-26     2007-12-21   
3          222715.39       0.0                  0.0 2010-05-20     2012-04-03   
4          112782.00       0.0                  0.0 2012-04-18     2014-05-19   

   SquareFootageofProject  DurationMonthsFloat  DurationMonths  
0                       0            11.269225              11  
1                       0            11.992033              11  
2                       0            34.793322              34  
3                       0            22.472741              22  
4                       1            25.002567              25

In [168]:
# Create column to join with the Construction PPI dataframe
Hospital_Projects_1['YearDF'] = Hospital_Projects_1['ProjStatusDate'].dt.year
Hospital_Projects_1['MonthDF'] = Hospital_Projects_1['ProjStatusDate'].dt.month
Hospital_Projects_1['Year_Month'] = Hospital_Projects_1['YearDF'].map(str) + '-' + Hospital_Projects_1['MonthDF'].map(str) 

Hospital_Projects_1.head()

FacilityNbr      FacilityName       Address1 AddrCity AddrState AddrZip  \
0        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
1        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
2        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
3        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   
4        10006  Orchard Hospital  240 Spruce St  Gridley        CA   95948   

    Latitude  Longitude    ProjectNbr                     ProjectName  \
0  39.367691 -121.68956    GS011843-1             NPC 2 RETROFIT WORK   
1  39.367691 -121.68956    SS013265-0              NPC 2 BULK MED GAS   
2  39.367691 -121.68956    GS050215-1         TOILET FINISHES UPGRADE   
3  39.367691 -121.68956    SS100827-0  ENTRANCE/LOBBY RECONFIGURATION   
4  39.367691 -121.68956  P-2012-00835            Chiller Installation   

      ...     RADCosts FixedEquipmentCosts   FileDate  ProjStatusDate  \
0     ...          0.0                 0.0 2001-07-19      2002-06-27   
1     ...          0.0                 0.0 2001-12-13      2002-12-13   
2     ...          0.0                 0.0 2005-01-26      2007-12-21   
3     ...          0.0                 0.0 2010-05-20      2012-04-03   
4     ...          0.0                 0.0 2012-04-18      2014-05-19   

   SquareFootageofProject DurationMonthsFloat DurationMonths  YearDF  MonthDF  \
0                       0           11.269225             11    2002        6   
1                       0           11.992033             11    2002       12   
2                       0           34.793322             34    2007       12   
3                       0           22.472741             22    2012        4   
4                       1           25.002567             25    2014        5   

   Year_Month  
0      2002-6  
1     2002-12  
2     2007-12  
3      2012-4  
4      2014-5  

[5 rows x 23 columns]

In [169]:
#Export dataframe to csv file
Hospital_Projects_1.to_csv('../Resources/Hospital_Projects_1.csv',index=False)

In [170]:
# Join DataFrames with construction cost and PPI information by year, respectively
Hospital_Projects_PPI = pd.merge(Hospital_Projects_1, Construction_PPI, on="Year_Month")
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName  ...    \
0                               Chiller Installation  ...     
1  ED Clean & Soiled Utility Room Renovation (151...  ...     
2  (151-703) WCR Hospital NICU Curtains and Flooring  ...     
3                        ED LCD MONITORS SEQUENCE II  ...     
4                                     4 E IDF CLOSET  ...     

  SquareFootageofProject DurationMonthsFloat  DurationMonths  YearDF  MonthDF  \
0                      1           25.002567              25    2014        5   
1                    315           27.335264              27    2014        5   
2                   4030           18.661574              18    2014        5   
3                      0           33.511982              33    2014        5   
4                     94           11.433500              11    2014        5   

  Year_Month      DATE  WPU801_NBD20190601  Year  Month  
0     2014-5  5/1/2014            86.60786  2014      5  
1     2014-5  5/1/2014            86.60786  2014      5  
2     2014-5  5/1/2014            86.60786  2014      5  
3     2014-5  5/1/2014            86.60786  2014      5  
4     2014-5  5/1/2014            86.60786  2014      5  

[5 rows x 27 columns]

In [171]:
# Calculate the 2019 project Construction Costs from older project Construction Costs
# Using PPI Index factor
Hospital_Projects_PPI['ConstructionCost_2019'] = (
    Hospital_Projects_PPI['ConstructionCosts'] * 100) / Hospital_Projects_PPI['WPU801_NBD20190601']
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName          ...            \
0                               Chiller Installation          ...             
1  ED Clean & Soiled Utility Room Renovation (151...          ...             
2  (151-703) WCR Hospital NICU Curtains and Flooring          ...             
3                        ED LCD MONITORS SEQUENCE II          ...             
4                                     4 E IDF CLOSET          ...             

  DurationMonthsFloat DurationMonths  YearDF  MonthDF  Year_Month      DATE  \
0           25.002567             25    2014        5      2014-5  5/1/2014   
1           27.335264             27    2014        5      2014-5  5/1/2014   
2           18.661574             18    2014        5      2014-5  5/1/2014   
3           33.511982             33    2014        5      2014-5  5/1/2014   
4           11.433500             11    2014        5      2014-5  5/1/2014   

  WPU801_NBD20190601  Year  Month  ConstructionCost_2019  
0           86.60786  2014      5          130221.437177  
1           86.60786  2014      5          177562.405999  
2           86.60786  2014      5          542406.890091  
3           86.60786  2014      5           27006.786682  
4           86.60786  2014      5           24190.645052  

[5 rows x 28 columns]

In [172]:
# Creating new "TotalCosts_2019" column because the original "TotalCosts" column was incorrect
#And to align with normalized ConstructionCosts_2019 information

Hospital_Projects_PPI['TotalCosts_2019'] = Hospital_Projects_PPI[
    'ConstructionCost_2019'] + Hospital_Projects_PPI['RADCosts'] + Hospital_Projects_PPI['FixedEquipmentCosts']
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName       ...         \
0                               Chiller Installation       ...          
1  ED Clean & Soiled Utility Room Renovation (151...       ...          
2  (151-703) WCR Hospital NICU Curtains and Flooring       ...          
3                        ED LCD MONITORS SEQUENCE II       ...          
4                                     4 E IDF CLOSET       ...          

  DurationMonths YearDF  MonthDF  Year_Month      DATE WPU801_NBD20190601  \
0             25   2014        5      2014-5  5/1/2014           86.60786   
1             27   2014        5      2014-5  5/1/2014           86.60786   
2             18   2014        5      2014-5  5/1/2014           86.60786   
3             33   2014        5      2014-5  5/1/2014           86.60786   
4             11   2014        5      2014-5  5/1/2014           86.60786   

   Year  Month  ConstructionCost_2019  TotalCosts_2019  
0  2014      5          130221.437177    130221.437177  
1  2014      5          177562.405999    177562.405999  
2  2014      5          542406.890091    542406.890091  
3  2014      5           27006.786682     27006.786682  
4  2014      5           24190.645052     48241.645052  

[5 rows x 29 columns]

In [173]:
# Calculate Construction Cost per Square Foot based on 2019 Cost information
Hospital_Projects_PPI['CCperSF'] = Hospital_Projects_PPI['ConstructionCost_2019'] / Hospital_Projects_PPI[
                                                        'SquareFootageofProject']
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName      ...      YearDF  \
0                               Chiller Installation      ...        2014   
1  ED Clean & Soiled Utility Room Renovation (151...      ...        2014   
2  (151-703) WCR Hospital NICU Curtains and Flooring      ...        2014   
3                        ED LCD MONITORS SEQUENCE II      ...        2014   
4                                     4 E IDF CLOSET      ...        2014   

  MonthDF  Year_Month      DATE  WPU801_NBD20190601  Year Month  \
0       5      2014-5  5/1/2014            86.60786  2014     5   
1       5      2014-5  5/1/2014            86.60786  2014     5   
2       5      2014-5  5/1/2014            86.60786  2014     5   
3       5      2014-5  5/1/2014            86.60786  2014     5   
4       5      2014-5  5/1/2014            86.60786  2014     5   

   ConstructionCost_2019  TotalCosts_2019       CCperSF  
0          130221.437177    130221.437177  1.302214e+05  
1          177562.405999    177562.405999  5.636902e+02  
2          542406.890091    542406.890091  1.345923e+02  
3           27006.786682     27006.786682           inf  
4           24190.645052     48241.645052  2.573473e+02  

[5 rows x 30 columns]

In [174]:
# Calculate Construction Cost per Square Foot based on 2019 Cost information
Hospital_Projects_PPI['TCperSF'] = Hospital_Projects_PPI['TotalCosts_2019'] / Hospital_Projects_PPI[
                                                        'SquareFootageofProject']
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName      ...      MonthDF  \
0                               Chiller Installation      ...            5   
1  ED Clean & Soiled Utility Room Renovation (151...      ...            5   
2  (151-703) WCR Hospital NICU Curtains and Flooring      ...            5   
3                        ED LCD MONITORS SEQUENCE II      ...            5   
4                                     4 E IDF CLOSET      ...            5   

  Year_Month      DATE  WPU801_NBD20190601  Year Month ConstructionCost_2019  \
0     2014-5  5/1/2014            86.60786  2014     5         130221.437177   
1     2014-5  5/1/2014            86.60786  2014     5         177562.405999   
2     2014-5  5/1/2014            86.60786  2014     5         542406.890091   
3     2014-5  5/1/2014            86.60786  2014     5          27006.786682   
4     2014-5  5/1/2014            86.60786  2014     5          24190.645052   

   TotalCosts_2019       CCperSF       TCperSF  
0    130221.437177  1.302214e+05  1.302214e+05  
1    177562.405999  5.636902e+02  5.636902e+02  
2    542406.890091  1.345923e+02  1.345923e+02  
3     27006.786682           inf           inf  
4     48241.645052  2.573473e+02  5.132090e+02  

[5 rows x 31 columns]

In [175]:
Hospital_Projects_PPI.count()

FacilityNbr               14367
FacilityName              14367
Address1                  14367
AddrCity                  14367
AddrState                 14367
AddrZip                   14367
Latitude                  14367
Longitude                 14367
ProjectNbr                14367
ProjectName               14364
Description               14367
KindofProject             14367
ConstructionCosts         14367
RADCosts                  14367
FixedEquipmentCosts       14367
FileDate                  14367
ProjStatusDate            14367
SquareFootageofProject    14367
DurationMonthsFloat       14367
DurationMonths            14367
YearDF                    14367
MonthDF                   14367
Year_Month                14367
DATE                      14367
WPU801_NBD20190601        14367
Year                      14367
Month                     14367
ConstructionCost_2019     14367
TotalCosts_2019           14367
CCperSF                   14332
TCperSF                   14355
dtype: i

In [176]:
# Remove following columns
Hospital_Projects_PPI = Hospital_Projects_PPI.drop(columns=["YearDF","MonthDF","Year","Month","Year_Month","DATE",
                                                            "ConstructionCosts", "DurationMonthsFloat"])
Hospital_Projects_PPI.head()

FacilityNbr                                  FacilityName  \
0        10006                              Orchard Hospital   
1        10072     Kaiser Foundation Hospital - Walnut Creek   
2        10072     Kaiser Foundation Hospital - Walnut Creek   
3        10081  John Muir Medical Center-Walnut Creek Campus   
4        10081  John Muir Medical Center-Walnut Creek Campus   

                   Address1      AddrCity AddrState AddrZip   Latitude  \
0             240 Spruce St       Gridley        CA   95948  39.367691   
1        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
2        1425 S Main Street  Walnut Creek        CA   94596  37.891754   
3  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   
4  1601 Ygnacio Valley Road  Walnut Creek        CA   94598  37.914182   

    Longitude     ProjectNbr  \
0 -121.689560   P-2012-00835   
1 -122.059532   P-2012-00251   
2 -122.059532   P-2012-02244   
3 -122.042134     SS111575-0   
4 -122.042134  S131258-07-00   

                                         ProjectName      ...       \
0                               Chiller Installation      ...        
1  ED Clean & Soiled Utility Room Renovation (151...      ...        
2  (151-703) WCR Hospital NICU Curtains and Flooring      ...        
3                        ED LCD MONITORS SEQUENCE II      ...        
4                                     4 E IDF CLOSET      ...        

  FixedEquipmentCosts   FileDate  ProjStatusDate  SquareFootageofProject  \
0                 0.0 2012-04-18      2014-05-19                       1   
1                 0.0 2012-02-08      2014-05-20                     315   
2                 0.0 2012-10-22      2014-05-13                    4030   
3                 0.0 2011-08-04      2014-05-20                       0   
4             24051.0 2013-06-06      2014-05-20                      94   

  DurationMonths WPU801_NBD20190601  ConstructionCost_2019  TotalCosts_2019  \
0             25           86.60786          130221.437177    130221.437177   
1             27           86.60786          177562.405999    177562.405999   
2             18           86.60786          542406.890091    542406.890091   
3             33           86.60786           27006.786682     27006.786682   
4             11           86.60786           24190.645052     48241.645052   

        CCperSF       TCperSF  
0  1.302214e+05  1.302214e+05  
1  5.636902e+02  5.636902e+02  
2  1.345923e+02  1.345923e+02  
3           inf           inf  
4  2.573473e+02  5.132090e+02  

[5 rows x 23 columns]

In [177]:
#Create new dataframe that only includes projects with total size of atleast 100 square feet
Hospital_Projects_PPI = Hospital_Projects_PPI[(Hospital_Projects_PPI['SquareFootageofProject'] >= 100)]
Hospital_Projects_PPI.count()

FacilityNbr               7786
FacilityName              7786
Address1                  7786
AddrCity                  7786
AddrState                 7786
AddrZip                   7786
Latitude                  7786
Longitude                 7786
ProjectNbr                7786
ProjectName               7783
Description               7786
KindofProject             7786
RADCosts                  7786
FixedEquipmentCosts       7786
FileDate                  7786
ProjStatusDate            7786
SquareFootageofProject    7786
DurationMonths            7786
WPU801_NBD20190601        7786
ConstructionCost_2019     7786
TotalCosts_2019           7786
CCperSF                   7786
TCperSF                   7786
dtype: int64

In [178]:
# #Create new dataframe that only excludes projects with construction cost of atleast $100,000
Hospital_Projects_PPI = Hospital_Projects_PPI[(Hospital_Projects_PPI['ConstructionCost_2019'] >= 100000)]
Hospital_Projects_PPI.count()

FacilityNbr               3946
FacilityName              3946
Address1                  3946
AddrCity                  3946
AddrState                 3946
AddrZip                   3946
Latitude                  3946
Longitude                 3946
ProjectNbr                3946
ProjectName               3943
Description               3946
KindofProject             3946
RADCosts                  3946
FixedEquipmentCosts       3946
FileDate                  3946
ProjStatusDate            3946
SquareFootageofProject    3946
DurationMonths            3946
WPU801_NBD20190601        3946
ConstructionCost_2019     3946
TotalCosts_2019           3946
CCperSF                   3946
TCperSF                   3946
dtype: int64

In [179]:
Hospital_Projects_PPI.dtypes

FacilityNbr                        int64
FacilityName                      object
Address1                          object
AddrCity                          object
AddrState                         object
AddrZip                           object
Latitude                         float64
Longitude                        float64
ProjectNbr                        object
ProjectName                       object
Description                       object
KindofProject                     object
RADCosts                         float64
FixedEquipmentCosts              float64
FileDate                  datetime64[ns]
ProjStatusDate            datetime64[ns]
SquareFootageofProject             int64
DurationMonths                     int32
WPU801_NBD20190601               float64
ConstructionCost_2019            float64
TotalCosts_2019                  float64
CCperSF                          float64
TCperSF                          float64
dtype: object

In [180]:
#export dataframe to csv file
Hospital_Projects_PPI.to_csv('../Resources/Hospital_Projects_PPI.csv',index=False)

In [181]:
# Remove following columns
Hospital_Projects_PPI2 = Hospital_Projects_PPI.drop(columns=["WPU801_NBD20190601", "RADCosts", 
                                                            "FixedEquipmentCosts"])
Hospital_Projects_PPI2.head()

FacilityNbr                               FacilityName            Address1  \
1        10072  Kaiser Foundation Hospital - Walnut Creek  1425 S Main Street   
2        10072  Kaiser Foundation Hospital - Walnut Creek  1425 S Main Street   
6        10268    Kaiser Foundation Hospital - San Rafael    99 Montecillo Rd   
7        10268    Kaiser Foundation Hospital - San Rafael    99 Montecillo Rd   
8        10273                     Marin General Hospital      250 Bon Air Rd   

       AddrCity AddrState AddrZip   Latitude   Longitude    ProjectNbr  \
1  Walnut Creek        CA   94596  37.891754 -122.059532  P-2012-00251   
2  Walnut Creek        CA   94596  37.891754 -122.059532  P-2012-02244   
6    San Rafael        CA   94903  38.003963 -122.555367  P-2012-01448   
7    San Rafael        CA   94903  38.003963 -122.555367  P-2012-02755   
8     Greenbrae        CA   94904  37.947544 -122.535454  P-2012-01754   

                                         ProjectName  \
1  ED Clean & Soiled Utility Room Renovation (151...   
2  (151-703) WCR Hospital NICU Curtains and Flooring   
6                    5th Floor Reception Ara Upgrade   
7        SRF Hospital 3rd Floor Restroom ADA Upgrade   
8                       MGH Lab Analyzer Replacement   

                                         Description KindofProject   FileDate  \
1  ED existing soiled utility and clean holding r...       Remodel 2012-02-08   
2  Providing new curtains and curtain tracks, and...       Remodel 2012-10-22   
6  The scope of work for this project will includ...       Remodel 2012-07-10   
7  Relocate existing sinks & Accessories for ADA ...       Remodel 2012-12-26   
8  Installation of new analyzers to replace exist...       Remodel 2012-08-22   

  ProjStatusDate  SquareFootageofProject  DurationMonths  \
1     2014-05-20                     315              27   
2     2014-05-13                    4030              18   
6     2014-05-19                     400              22   
7     2014-05-07                     154              16   
8     2014-05-13                    1184              20   

   ConstructionCost_2019  TotalCosts_2019     CCperSF     TCperSF  
1          177562.405999    177562.405999  563.690178  563.690178  
2          542406.890091    542406.890091  134.592280  134.592280  
6          280524.192608    280524.192608  701.310482  701.310482  
7          110625.063360    110625.063360  718.344567  718.344567  
8          315245.059744    752095.059744  266.254273  635.215422

In [182]:
Hospital_Projects_PPI2.count()

FacilityNbr               3946
FacilityName              3946
Address1                  3946
AddrCity                  3946
AddrState                 3946
AddrZip                   3946
Latitude                  3946
Longitude                 3946
ProjectNbr                3946
ProjectName               3943
Description               3946
KindofProject             3946
FileDate                  3946
ProjStatusDate            3946
SquareFootageofProject    3946
DurationMonths            3946
ConstructionCost_2019     3946
TotalCosts_2019           3946
CCperSF                   3946
TCperSF                   3946
dtype: int64

In [183]:
#export dataframe to csv file.  Similar to Hospital_Projects_PPI except eliminates:
#PPI index factor, RADCosts, and FixedEquipmentCosts.
Hospital_Projects_PPI2.to_csv('../Resources/Hospital_Projects_PPI2.csv',index=False)

In [184]:
#Create new dataframe that increases parameter for projects with total size of atleast 1000 square feet
# in attempt to screen data for better quality
Hospital_Projects_PPI3 = Hospital_Projects_PPI2[(Hospital_Projects_PPI2['SquareFootageofProject'] >= 100)]
Hospital_Projects_PPI3.count()

FacilityNbr               3946
FacilityName              3946
Address1                  3946
AddrCity                  3946
AddrState                 3946
AddrZip                   3946
Latitude                  3946
Longitude                 3946
ProjectNbr                3946
ProjectName               3943
Description               3946
KindofProject             3946
FileDate                  3946
ProjStatusDate            3946
SquareFootageofProject    3946
DurationMonths            3946
ConstructionCost_2019     3946
TotalCosts_2019           3946
CCperSF                   3946
TCperSF                   3946
dtype: int64

In [185]:
# Hospital_Projects_PPI = Hospital_Projects_PPI.drop(columns=["FacilityNbr","FacilityName", "Address1", "AddrCity",
#                                                                "AddrState", "AddrZip","Latitude", "Longitude",
#                                                                "ProjectNbr","ProjectName","Description",
#                                                                "ProjStatusDate","FileDate","CCperSF, TCperSF" ])

# Hospital_Projects_PPI.count()

In [186]:
Hospital_Projects_PPI3_Maint=Hospital_Projects_PPI3[Hospital_Projects_PPI3.KindofProject == 'Maintenance']
Hospital_Projects_PPI3_Maint.count()


FacilityNbr               494
FacilityName              494
Address1                  494
AddrCity                  494
AddrState                 494
AddrZip                   494
Latitude                  494
Longitude                 494
ProjectNbr                494
ProjectName               494
Description               494
KindofProject             494
FileDate                  494
ProjStatusDate            494
SquareFootageofProject    494
DurationMonths            494
ConstructionCost_2019     494
TotalCosts_2019           494
CCperSF                   494
TCperSF                   494
dtype: int64

In [187]:
#export dataframe to csv file
Hospital_Projects_PPI3_Maint.to_csv('../Resources/Hospital_Projects_PPI3_Maint.csv',index=False)

In [188]:
Hospital_Projects_PPI3_Add=Hospital_Projects_PPI3[Hospital_Projects_PPI3.KindofProject == 'Addition']
Hospital_Projects_PPI3_Add.count()

FacilityNbr               40
FacilityName              40
Address1                  40
AddrCity                  40
AddrState                 40
AddrZip                   40
Latitude                  40
Longitude                 40
ProjectNbr                40
ProjectName               40
Description               40
KindofProject             40
FileDate                  40
ProjStatusDate            40
SquareFootageofProject    40
DurationMonths            40
ConstructionCost_2019     40
TotalCosts_2019           40
CCperSF                   40
TCperSF                   40
dtype: int64

In [189]:
#export dataframe to csv file
Hospital_Projects_PPI3_Add.to_csv('../Resources/Hospital_Projects_PPI3_Add.csv',index=False)

In [190]:
Hospital_Projects_PPI3_Rem=Hospital_Projects_PPI3[Hospital_Projects_PPI3.KindofProject == 'Remodel']
Hospital_Projects_PPI3_Rem.count()


FacilityNbr               3349
FacilityName              3349
Address1                  3349
AddrCity                  3349
AddrState                 3349
AddrZip                   3349
Latitude                  3349
Longitude                 3349
ProjectNbr                3349
ProjectName               3346
Description               3349
KindofProject             3349
FileDate                  3349
ProjStatusDate            3349
SquareFootageofProject    3349
DurationMonths            3349
ConstructionCost_2019     3349
TotalCosts_2019           3349
CCperSF                   3349
TCperSF                   3349
dtype: int64

In [191]:
#export dataframe to csv file
Hospital_Projects_PPI3_Rem.to_csv('../Resources/Hospital_Projects_PPI3_Rem.csv',index=False)

In [192]:
Hospital_Projects_PPI3_New=Hospital_Projects_PPI3[Hospital_Projects_PPI3.KindofProject == 'New Building']
Hospital_Projects_PPI3_New.count()

FacilityNbr               63
FacilityName              63
Address1                  63
AddrCity                  63
AddrState                 63
AddrZip                   63
Latitude                  63
Longitude                 63
ProjectNbr                63
ProjectName               63
Description               63
KindofProject             63
FileDate                  63
ProjStatusDate            63
SquareFootageofProject    63
DurationMonths            63
ConstructionCost_2019     63
TotalCosts_2019           63
CCperSF                   63
TCperSF                   63
dtype: int64

In [193]:
#export dataframe to csv file
Hospital_Projects_PPI3_New.to_csv('../Resources/Hospital_Projects_PPI3_New.csv',index=False)